# GAN on Fashion MNIST
This notebook applies a GAN model to create other fashion image from the fashion MNIST dataset.

### Install required packages

It is recommended to install the following packages: keras==2.1.2, keras_adversarial.
Examples on how to install these dependencies are below:

In [0]:
!pip install --force-reinstall keras==2.1.2 #install keras compatible with keras_adversarial

  Using cached https://files.pythonhosted.org/packages/68/89/58ee5f56a9c26957d97217db41780ebedca3154392cb903c3f8a08a52208/Keras-2.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/50/a552a5aff252ae915f522e44642bb49a7b7b31677f9580cfd11bcc869976/scipy-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e5/e6/c3fdc53aed9fa19d6ff3abf97dfad768ae3afce1b7431f7500000816bda5/numpy-1.17.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing instal

In [0]:
!git clone https://github.com/bstriner/keras_adversarial.git #clone keras_adversarial

fatal: destination path 'keras_adversarial' already exists and is not an empty directory.


In [0]:
!cd keras_adversarial && python setup.py install #install keras_adversarial
#you may restart the runtime 

running install
running bdist_egg
running egg_info
writing keras_adversarial.egg-info/PKG-INFO
writing dependency_links to keras_adversarial.egg-info/dependency_links.txt
writing requirements to keras_adversarial.egg-info/requires.txt
writing top-level names to keras_adversarial.egg-info/top_level.txt
writing manifest file 'keras_adversarial.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/keras_adversarial
copying build/lib/keras_adversarial/adversarial_model.py -> build/bdist.linux-x86_64/egg/keras_adversarial
copying build/lib/keras_adversarial/legacy.py -> build/bdist.linux-x86_64/egg/keras_adversarial
creating build/bdist.linux-x86_64/egg/keras_adversarial/backend
copying build/lib/keras_adversarial/backend/theano_backend.py -> build/bdist.linux-x86_64/egg/keras_adversarial/backend
copying build/lib/keras_adversarial/backend/tensorflow_monke

### Import packages:

In [1]:
import pandas as pd # for data handling
import numpy as np # for linear algebra
import matplotlib.pyplot as plt #for visualisation
import keras #for neural networks

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [0]:
#for neural networks
from keras_adversarial import AdversarialModel, simple_gan, gan_targets
from keras_adversarial import normal_latent_sampling, AdversarialOptimizerSimultaneous
from keras_adversarial.legacy import fit
import keras.backend as K
from keras.layers import Conv2D, Flatten, Activation, Dense, UpSampling2D, Reshape, BatchNormalization, AveragePooling2D
from keras.layers import BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.initializers import TruncatedNormal
from keras.models import Sequential
from keras_adversarial.image_grid_callback import ImageGridCallback

### Prepare data

In [3]:
# get fashion mnist data
(x_train,y_train), (x_test,y_test) = keras.datasets.fashion_mnist.load_data()

# show shapes of tensors
print("x_train shape:", x_train.shape, ", y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, ", y_test shape:", y_test.shape)

# get number of classes
nClasses = len(np.unique(y_train)) # number of output classes
print("Number of classes: ", nClasses)

# normalize grayscale pixel values (0-255) to (0,1)
x_train = x_train.astype('float32')/255 # normalized training inputs
x_test = x_test.astype('float32')/255 # normalized test inputs

# show shapes of re-shaped tensors
print("x_train shape:", x_train.shape, ", y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, ", y_test shape:", y_test.shape)

x_train shape: (60000, 28, 28) , y_train shape: (60000,)
x_test shape: (10000, 28, 28) , y_test shape: (10000,)
Number of classes:  10
x_train shape: (60000, 28, 28) , y_train shape: (60000,)
x_test shape: (10000, 28, 28) , y_test shape: (10000,)


In [0]:
latent_dim = 100 #dimention of the output 

### Build generator model: Dense-Conv1-Conv2-Conv3

In [0]:
def model_generator():
    model_g = keras.Sequential([
        #first fully connected
        Dense(3136,  input_shape=(100,), kernel_initializer=TruncatedNormal(stddev=0.02), bias_initializer=TruncatedNormal(stddev=0.02)),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        Reshape([56, 56, 1]),

        #convolutional layer 1
        Conv2D(50, kernel_size = (3,3), strides=(2, 2), kernel_initializer=TruncatedNormal(stddev=0.02),bias_initializer=TruncatedNormal(stddev=0.02),  padding="same"),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        UpSampling2D(size=(2, 2)),

        #convolutional layer 2
        Conv2D(25, kernel_size = (3,3), strides=(2, 2), kernel_initializer=TruncatedNormal(stddev=0.02),bias_initializer=TruncatedNormal(stddev=0.02),  padding="same"),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        UpSampling2D(size=(2, 2)),

        #convolutional layer 3
        Conv2D(1, kernel_size=(1,1), strides=(2, 2), padding="same", kernel_initializer=TruncatedNormal(stddev=0.02), bias_initializer=TruncatedNormal(stddev=0.02), activation ="sigmoid")],
        name="generator")
    return model_g



### Build discriminator model with batch normalization: Conv1-Conv2-Dense1-Dense2

In [0]:
def model_discriminator():
    model_d = keras.Sequential([
        #convolutional layer 1
        Conv2D(32, kernel_size = (5,5), strides=(1, 1), kernel_initializer=TruncatedNormal(stddev=0.02), padding="same", input_shape=(28, 28, 1)),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),
        
        #convolutional layer 2
        Conv2D(64, kernel_size = (5,5), strides=(1, 1), kernel_initializer=TruncatedNormal(stddev=0.02), padding="same"),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),
        
        #first fully connected
        Flatten(),
        Dense(1024,  kernel_initializer=TruncatedNormal(stddev=0.02)),
        Activation('relu'),
        
        #second fully connected
        Dense(1, kernel_initializer=TruncatedNormal(stddev=0.02)),
        Activation('sigmoid')],
        name="discriminator")
    return model_d




In [0]:
def generator_sampler(latent_dim, generator): #data sampling for generator inputs
    def fun():
        zsamples = np.random.normal(0, 1, size=(10 * 10, latent_dim)) #sample data from normal distribution with mean 0 and standard deviation 1
        gen = dim_ordering_unfix(generator.predict(zsamples))
        return gen.reshape((10, 10, 28, 28))

    return fun

In [8]:
# generator (z -> x)
generator = model_generator()
# discriminator (x -> y)
discriminator = model_discriminator()
gan = simple_gan(generator, discriminator, normal_latent_sampling((latent_dim,)))






Instructions for updating:
keep_dims is deprecated, use keepdims instead



In [9]:
#NNs' architecture
generator.summary()
discriminator.summary()
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3136)              316736    
_________________________________________________________________
batch_normalization_1 (Batch (None, 3136)              12544     
_________________________________________________________________
activation_1 (Activation)    (None, 3136)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 56, 56, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 50)        500       
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 50)        200       
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 50)        0         
__________

In [10]:
model = AdversarialModel(base_model=gan, #build the model
                             player_params=[generator.trainable_weights, discriminator.trainable_weights],
                             player_names=["generator", "discriminator"])
model.adversarial_compile(adversarial_optimizer=AdversarialOptimizerSimultaneous(), #compile the model
                              player_optimizers=[Adam(0.0003, decay=1e-4), Adam(0.0003, decay=1e-4)],
                              loss='binary_crossentropy')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
generator_cb = ImageGridCallback("output/gan_convolutional/epoch-{:03d}.png",
                                     generator_sampler(latent_dim, generator)) #save the output after each epoch

In [0]:
#for the image dimension ordering convention  
def dim_ordering_fix(x):
    if K.image_dim_ordering() == 'th':
        return x
    else:
        return np.transpose(x, (0, 2, 3, 1))
    
def dim_ordering_unfix(x):
    if K.image_dim_ordering() == 'th':
        return x
    else:
        return np.transpose(x, (0, 3, 1, 2))
      
def dim_ordering_shape(input_shape):
    if K.image_dim_ordering() == 'th':
        return input_shape
    else:
        return (input_shape[1], input_shape[2], input_shape[0])

In [0]:
xtrain = dim_ordering_fix(x_train.reshape((-1, 1, 28, 28)))
xtest = dim_ordering_fix(x_test.reshape((-1, 1, 28, 28)))
y = gan_targets(xtrain.shape[0])

In [15]:
%time
ytest = gan_targets(xtest.shape[0])
history = model.fit(x=xtrain, y=y, validation_data=(xtest, ytest), callbacks=[generator_cb], nb_epoch=5,
                        batch_size=100) #training the DCGAN
df = pd.DataFrame(history.history) #convert to dataframe
df.to_csv("output/gan_convolutional/history.csv") #save the loss history 

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.78 µs
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
  100/60000 [..............................] - ETA: 1:07 - loss: 11.0530 - generator_loss: 11.0232 - generator_yfake_loss: 5.3271 - generator_yreal_loss: 5.6961 - discriminator_loss: 0.0298 - discriminator_yfake_loss: 0.0128 - discriminator_yreal_loss: 0.0170

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


60000/60000 [==============================] - 66s 1ms/step - loss: 10.5978 - generator_loss: 10.4289 - generator_yfake_loss: 4.9500 - generator_yreal_loss: 5.4789 - discriminator_loss: 0.1689 - discriminator_yfake_loss: 0.0810 - discriminator_yreal_loss: 0.0879 - val_loss: 25.6122 - val_generator_loss: 14.9260 - val_generator_yfake_loss: 14.9224 - val_generator_yreal_loss: 0.0036 - val_discriminator_loss: 10.6862 - val_discriminator_yfake_loss: 1.5074e-06 - val_discriminator_yreal_loss: 10.6862
Epoch 2/5
60000/60000 [==============================] - 66s 1ms/step - loss: 10.3852 - generator_loss: 10.1806 - generator_yfake_loss: 4.6934 - generator_yreal_loss: 5.4872 - discriminator_loss: 0.2045 - discriminator_yfake_loss: 0.0966 - discriminator_yreal_loss: 0.1079 - val_loss: 17.9521 - val_generator_loss: 11.6779 - val_generator_yfake_loss: 0.0186 - val_generator_yreal_loss: 11.6593 - val_discriminator_loss: 6.2742 - val_discriminator_yfake_loss: 6.2738 - val_discriminator_yreal_loss: 4

In [16]:
print(history.history.keys())

dict_keys(['val_loss', 'val_generator_loss', 'val_generator_yfake_loss', 'val_generator_yreal_loss', 'val_discriminator_loss', 'val_discriminator_yfake_loss', 'val_discriminator_yreal_loss', 'loss', 'generator_loss', 'generator_yfake_loss', 'generator_yreal_loss', 'discriminator_loss', 'discriminator_yfake_loss', 'discriminator_yreal_loss'])


In [0]:
#save models' weights
generator.save("output/gan_convolutional/generator.h5")
discriminator.save("output/gan_convolutional/discriminator.h5")